In [1]:
import numpy as np
%autosave 600

Autosaving every 600 seconds


In [2]:
from scikits.odes.odeint import odeint

In [3]:
help(odeint)

Help on function odeint in module scikits.odes.odeint:

odeint(rhsfun, tout, y0, method='bdf', **options)
    Integrate a system of ordinary differential equations.
    *odeint* is a wrapper around the ode class, as a confenience function to
    quickly integrate a system of ode.
    Solves the initial value problem for stiff or non-stiff systems
    of first order ode's:
    
        rhs = dy/dt = fun(t, y)
    
    where y can be a vector, then rhsfun must be a function computing rhs with
    signature:
    
        rhsfun(t, y, rhs)
    
    storing the computated dy/dt in the rhs array passed to the function
    
    Parameters
    ----------
    rhsfun : callable(t, y, out)
        Computes the derivative at dy/dt in terms of t and y, and stores in out
    y0 : array
        Initial condition on y (can be a vector).
    t : array
        A sequence of time points for which to solve for y.  The initial
        value point should be the first element of this sequence.
    method : s

In [4]:
#from scipy.integrate import solve_ivp

For the following the Cython module must be installed: pip3 install Cython

In [5]:
%load_ext Cython

/usr/local/lib/python3.5/dist-packages/Cython/Distutils/old_build_ext.py:30: UserWarning: Cython.Distutils.old_build_ext does not properly handle dependencies and is deprecated.
  "Cython.Distutils.old_build_ext does not properly handle dependencies "


In [6]:
def fun(t, y, out):
    mu = 50
    out[0] = y[1]
    out[1] =  mu * (1 - y[0] ** 2) * y[1] - y[0]

In [7]:
def jac(t, y, out):
    mu = 50
    out[:,:] = [
        [0, 1],
        [-2 * mu * y[0] * y[1] - 1, mu * (1 - y[0] ** 2)]]

In [8]:
%%cython
def fun_cython(double t, double[:] y, double [:] f):
    cdef double mu = 50
    f[0] = y[1]
    f[1] = mu * (1 - y[0] * y[0]) * y[1] - y[0]

In [18]:
%%cython -I /home/benny/git/odes/scikits/odes/sundials/ -I /usr/local/lib/python3.5/dist-packages/scikits.odes-2.3.0.dev0-py3.5-linux-x86_64.egg/scikits/odes/sundials/
### UPDATE ABOVE -I flag to installed odes/sundials sources !!
import numpy as np
cimport numpy as np
from scikits.odes.sundials.cvode cimport CV_RhsFunction
    
#scikits.odes allows cython functions only if derived from correct class
cdef class RhsFunction(CV_RhsFunction):
    cpdef int evaluate(self, double t,
                       np.ndarray[double, ndim=1] y,
                       np.ndarray[double, ndim=1] f,
                       object userdata = None) except? -1:
        cdef double mu = 50
        f[0] = y[1]
        f[1] = mu * (1 - y[0] * y[0]) * y[1] - y[0]
        return 0

In [10]:
y0 = [2, 0]
outtimes = np.linspace(0, 1000, 101)

In [11]:
sol1 = odeint(fun, outtimes, y0, method='cvode')

In [12]:
stats = %prun -r -q odeint(fun, outtimes, y0, method='cvode')

In [13]:
stats.print_stats(20)

         15637 function calls in 0.062 seconds

   Ordered by: internal time
   List reduced from 47 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.045    0.045    0.062    0.062 {method 'solve' of 'scikits.odes.sundials.cvode.CVODE' objects}
    15534    0.017    0.000    0.017    0.000 <ipython-input-6-d6e24033dc3f>:1(fun)
        1    0.000    0.000    0.062    0.062 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 inspect.py:2065(_signature_from_function)
        2    0.000    0.000    0.000    0.000 copy.py:67(copy)
        1    0.000    0.000    0.000    0.000 inspect.py:1052(getfullargspec)
        1    0.000    0.000    0.062    0.062 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 ode.py:220(__init__)
        1    0.000    0.000    0.000    0.000 {built-in method _warnings.warn}
        1    0.000    0.000    0.000    0.000 inspect.py:1027(getargspec)
        

In [19]:
sol2 = odeint(RhsFunction(), outtimes, y0, method='cvode')

In [20]:
stats = %prun -r -q odeint(RhsFunction(), outtimes, y0, method='cvode')

In [21]:
stats.print_stats(20)

         49 function calls in 0.042 seconds

   Ordered by: internal time
   List reduced from 27 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.041    0.041    0.042    0.042 {method 'solve' of 'scikits.odes.sundials.cvode.CVODE' objects}
        2    0.000    0.000    0.000    0.000 copy.py:67(copy)
        1    0.000    0.000    0.042    0.042 {built-in method builtins.exec}
        1    0.000    0.000    0.042    0.042 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 ode.py:220(__init__)
        4    0.000    0.000    0.000    0.000 <string>:12(__new__)
        2    0.000    0.000    0.000    0.000 {built-in method numpy.core.multiarray.array}
        1    0.000    0.000    0.042    0.042 odeint.py:13(odeint)
        1    0.000    0.000    0.000    0.000 {method 'match' of '_sre.SRE_Pattern' objects}
        2    0.000    0.000    0.000    0.000 numeric.py:414(asarray)
        1    0.000    0.0

In [17]:
#verify the two solutions are equal
for t, u1, u2 in zip(sol1.values.t, sol1.values.y,  sol2.values.y):
    print('{0:>4.0f} {1:15.6g} {2:15.6g}'.format(t, u1[0], u2[0]) )

   0               2               2
  10         1.85832         1.85832
  20         1.69338         1.69338
  30          1.4849          1.4849
  40         1.09322         1.09322
  50        -1.87714        -1.87714
  60        -1.71581        -1.71581
  70        -1.51498        -1.51498
  80        -1.18599        -1.18599
  90         1.89563         1.89563
 100         1.73766         1.73766
 110         1.54362         1.54362
 120         1.24882         1.24882
 130         -1.9138         -1.9138
 140        -1.75896        -1.75896
 150        -1.57099        -1.57099
 160         -1.2998         -1.2998
 170         1.93167         1.93167
 180         1.77978         1.77978
 190          1.5973          1.5973
 200         1.34404         1.34404
 210        -1.94927        -1.94927
 220        -1.80014        -1.80014
 230        -1.62265        -1.62265
 240        -1.38373        -1.38373
 250          1.9666          1.9666
 260         1.82008         1.82008
 